# Lab | Handling Data Imbalance in Classification Models

For this lab and in the next lessons we will build a model on a customer churn binary classification problem. You will be using `files_for_lab/Customer-Churn.csv` file.

### Scenario

You are working as an analyst with this internet service provider. You are provided with this historical data about your company's customers and their churn trends. Your task is to build a machine learning model that will help the company identify customers that are more likely to default/churn and thus prevent losses from such customers.

### Instructions

In this lab, we will first take a look at the degree of imbalance in the data and correct it using the techniques we learned in the class.

Here is the list of steps to be followed (building a simple model without balancing the data):

- Import the required libraries and modules that you would need.
- Read that data into Python and call the dataframe `churnData`.
- Check the datatypes of all the columns in the data. You would see that the column `TotalCharges` is object type. Convert this column into numeric type using `pd.to_numeric` function.
- Check for null values in the dataframe. Replace the null values.
- Use the following features: `tenure`, `SeniorCitizen`, `MonthlyCharges` and `TotalCharges`:
  - Scale the features either by using normalizer or a standard scaler.
  - Split the data into a training set and a test set.
  - Fit a logistic regression model on the training data.
  - Check the accuracy on the test data.

In [1]:
#Import the required libraries and modules 
import pandas as pd
import numpy as np
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score,mean_squared_error, mean_absolute_error
import statsmodels.api as stm
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
#Read that data into Python and call the dataframe 'churnData'.
churnData = pd.read_csv('./files_for_lab/Customer-Churn.csv')
churnData.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No,Yes,No,No,No,No,Month-to-month,29.85,29.85,No
1,Male,0,No,No,34,Yes,Yes,No,Yes,No,No,No,One year,56.95,1889.5,No
2,Male,0,No,No,2,Yes,Yes,Yes,No,No,No,No,Month-to-month,53.85,108.15,Yes
3,Male,0,No,No,45,No,Yes,No,Yes,Yes,No,No,One year,42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,No,No,No,No,No,Month-to-month,70.70,151.65,Yes


In [3]:
#- Check the datatypes of all the columns in the data.
churnData.dtypes
#  Convert 'TotalCharges' column into numeric type using `pd.to_numeric` function.

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [4]:
# Convert 'TotalCharges' column into numeric type using `pd.to_numeric` function.
# there are some values in the dataset that cannot be parsed as numeric with 'pd.to_numeric'
# Define a function to replace those values with NaN values.
def nan_maker(x):
    try:
        x = pd.to_numeric(x)
    except: 
        x = np.nan
    return x

churnData['TotalCharges'] = churnData['TotalCharges'].apply(lambda x: nan_maker(x))
churnData['TotalCharges'].dtype

dtype('float64')

In [5]:
# Check for null values in the dataframe. 
churn_nulls = pd.DataFrame(churnData.isna().sum()).reset_index()
churn_nulls.columns = ['Columns', 'Nulls']
# Replace the null values with mean values for the column.
churnData['TotalCharges'].fillna(np.mean(churnData['TotalCharges']), inplace = True)
churn_nulls.iloc[14]

Columns    TotalCharges
Nulls                11
Name: 14, dtype: object

In [6]:
# Use the following features: `tenure`, `SeniorCitizen`, `MonthlyCharges` and `TotalCharges`:

X = churnData[['tenure','SeniorCitizen','MonthlyCharges','TotalCharges']]
y = churnData['Churn']  

In [7]:
# convert categorical labels to numeric for model output.
def yes_no_to_bin(x):
    if x.lower() == 'yes':
        x = 1
    else:
        x = 0
    return x
y = y.apply(lambda x: yes_no_to_bin(x))

In [8]:
# Split data into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size= 0.3, random_state = 32)

In [9]:
# Scale the features either by using normalizer or a standard scaler.
stand_scaler = StandardScaler().fit(X_train)
norm_scaler = MinMaxScaler().fit(X_train)
X_train_st_scaled = stand_scaler.transform(X_train)
X_train_n_scaled = norm_scaler.transform(X_train)
X_test_st_scaled = stand_scaler.transform(X_test)
X_test_n_scaled = norm_scaler.transform(X_test) 

In [10]:
# Fit a logistic regression model on the training data.
LogR1 = LogisticRegression().fit(X_train_n_scaled,y_train)
training1 = LogR1.predict(X_train)
train_score1 = r2_score(y_train, training1)
print('Training: ', train_score1)


Training:  -1.2512616862167292


In [11]:
# Check the accuracy on the test data.
testing1 = LogR1.predict(X_test_st_scaled)
test_score1 = r2_score(y_test,testing1)
print('Testing: ', test_score1)

Testing:  -0.40596025052373363



## **Note**: So far we have not balanced the data.

Managing imbalance in the dataset

- Check for the imbalance.
- Use the resampling strategies used in class for upsampling and downsampling to create a balance between the two classes.
- Each time fit the model and see how the accuracy of the model is.


In [12]:
# check for imbalance in the data.
churnData['Churn'].value_counts()

# the target data is somewhat imbalanced with 2.77 times more 'Churn' values of 'No' than 'Yes'


No     5174
Yes    1869
Name: Churn, dtype: int64

In [13]:
#Use the resampling strategies upsampling and downsampling to create a balance between the two classes.
# create separate datasets using the target categories
churnData_Yes = churnData[churnData['Churn'] == 'Yes']
churnData_No = churnData[churnData['Churn'] == 'No']
print(churnData_Yes.shape)
print(churnData_No.shape)

(1869, 16)
(5174, 16)


In [14]:
# upsample the minority class 'Yes'
churnData_Yes_oversample = resample(churnData_Yes, replace = True, n_samples = len(churnData_No))
churnData_upsampled = pd.concat([churnData_Yes_oversample, churnData_No], axis = 0)
print(churnData_upsampled.shape)
print(churnData_upsampled['Churn'].value_counts())


(10348, 16)
Yes    5174
No     5174
Name: Churn, dtype: int64


In [15]:
# split resampled data into training and testing data.
X_up = churnData_upsampled[['tenure','SeniorCitizen','MonthlyCharges','TotalCharges']]
y_up = churnData_upsampled['Churn']
y_up = y_up.apply(lambda x: yes_no_to_bin(x))

X_up_train, X_up_test, y_up_train, y_up_test = train_test_split(X_up, y_up,  test_size = 0.3, random_state = 32)

In [16]:
# fit a logistic regression model on the upsampled data
LogR_up = LogisticRegression().fit(X_up_train,y_up_train)
training_up = LogR_up.predict(X_up_train)
train_score_up = r2_score(y_up_train, training_up)
print('Training: ', train_score_up)

Training:  -0.05813979022621263


In [17]:
# Check the accuracy on the test data.
testing_up = LogR_up.predict(X_up_test)
test_score_up = r2_score(y_up_test,testing_up)
print('Testing: ', test_score_up)

Testing:  -0.06803567027402457


In [18]:
# downsample the majority class 'No'
churnData_No_undersample = resample(churnData_No, replace = False, n_samples = len(churnData_Yes))
churnData_downsampled = pd.concat([churnData_No_undersample, churnData_Yes], axis = 0)
print(churnData_downsampled.shape)
print(churnData_downsampled['Churn'].value_counts())


(3738, 16)
No     1869
Yes    1869
Name: Churn, dtype: int64


In [19]:
# split resampled data into training and testing data.
X_down = churnData_downsampled[['tenure','SeniorCitizen','MonthlyCharges','TotalCharges']]
y_down = churnData_downsampled['Churn']
y_down = y_down.apply(lambda x: yes_no_to_bin(x))

X_down_train, X_down_test, y_down_train, y_down_test = train_test_split(X_down, y_down,  test_size = 0.3, random_state = 32)

In [20]:
# fit a logistic regression model on the downsampled data
LogR_down = LogisticRegression().fit(X_down_train,y_down_train)
training_down = LogR_down.predict(X_down_train)
train_score_down = r2_score(y_down_train, training_down)
print('Training: ', train_score_down)

Training:  -0.05968319769368757


In [21]:
# Check the accuracy on the test data.
testing_down = LogR_down.predict(X_down_test)
test_score_down = r2_score(y_down_test,testing_down)
print('Testing: ', test_score_down)

Testing:  -0.02700228832951912
